# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders = pd.read_csv('../data/orders_sample.csv')
orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,563094,23152,2011,8,4,17,ivory sweetheart wire letter rack,2,2011-08-11 17:35:00,3.75,14525,United Kingdom,7.50
19996,578626,21592,2011,11,4,15,retrospot cigar box matches,24,2011-11-24 15:38:00,1.25,12709,Germany,30.00
19997,561073,23245,2011,7,7,15,set of 3 regency cake tins,16,2011-07-24 15:46:00,4.15,13408,United Kingdom,66.40
19998,576861,20674,2011,11,3,16,green polkadot bowl,12,2011-11-16 16:00:00,1.25,18041,United Kingdom,15.00


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [13]:
# your code here
customer_spent = orders.groupby(by='CustomerID')[['amount_spent']].sum()
customer_spent.head()

,amount_spent
CustomerID,
12347,149.90
12348,75.36
12349,100.09
12350,10.20
12352,126.48


How to select customers whose aggregated amount_spent is in a given quantile range?

In [32]:
customer_spent.quantile(np.arange(0,1.1, 0.25))

,amount_spent
0.00,0.390
0.25,19.905
0.50,48.000
0.75,112.290
1.00,12518.460


In [61]:
q90 = customer_spent.amount_spent.quantile(0.90)
q90

238.39999999999998

In [65]:
q75 = customer_spent.amount_spent.quantile(0.75)
q75

112.29000000000002

In [66]:
customer_spent.query(f'amount_spent > {q75}')

,amount_spent
CustomerID,
12347,149.90
12352,126.48
12356,161.88
12357,264.30
12359,324.85
...,...
18245,143.30
18251,252.00
18260,118.65


How to label selected customers as "VIP" or "Preferred"?

In [67]:
VIP = customer_spent.query(f'amount_spent > {q90}')
Preffered =  customer_spent.query(f'amount_spent > {q75}')

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [75]:
orders['Client'] = np.where(orders['amount_spent'] > q90, 'VIP',
                           np.where(orders['amount_spent'] > q75, 'Preferred', 
                                   np.where(orders['amount_spent'] < q75, 'NORMAL', 'NORMAL')))

In [152]:
client_type = ['VIP']
orders_vip = orders.loc[orders['Client'].isin(client_type)]
orders_vip.groupby('Country')[['Client']].count()

,Client
Country,
Australia,10
EIRE,8
Germany,5
Israel,1
Japan,1
Netherlands,11
Sweden,1
United Kingdom,100


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [153]:
# your code here
client_type = ['VIP', 'Preferred']
orders_vip = orders.loc[orders['Client'].isin(client_type)]
orders_vip.groupby('Country')[['Client']].count()

,Client
Country,
Australia,27
Austria,1
EIRE,23
Finland,1
France,3
Germany,13
Greece,1
Israel,2
Japan,7
